<a href="https://colab.research.google.com/github/MaharshSuryawala/Face-Detection-and-Facial-Expression-Recognition/blob/master/FaceDetectionAndExpressionRecognition_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
import os

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#import google.colab import drive
#drive.mount('/gdrive')

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/ML')

try:
  os.makedirs(local_download_path)
except: pass


In [0]:
# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'12efh4IZ9r54HgrnPULW0qgFk2MjIDR5A' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)

In [0]:
import numpy as np
from matplotlib import pyplot as plt
#py.init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
import math
from scipy.stats import norm
import time
import pandas as pd
import statsmodels.api as sm
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.models import Sequential
from matplotlib import pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from keras.utils.np_utils import to_categorical
%matplotlib inline

#from PIL import Image
#from resizeimage import resizeimage


/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.


In [0]:
%cd /root/ML

data = pd.read_csv('Train_Roll_Number.txt', sep=" ", header=None)
data1 = pd.read_csv('Train_RGB_Sketch.txt', sep=" ", header=None)
data2 = pd.read_csv('Train_Gender.txt', sep=" ", header=None)
data3 = pd.read_csv('Train_Expression.txt', sep=" ", header=None)

data.columns = ["filename", "subject"]
data1.columns = ["filename", "sketch"]
data2.columns = ["filename", "gender"]
data3.columns = ["filename", "expression"]

data['X'] = None
X = None

set_type = set()
set_shape = set()

for index, row in data.iterrows():
    # print(row["filename"], row["X"], row["subject"])
    
    im = cv2.imread(row["filename"])
    
    if row["filename"] == "1641010_Male_Neutral_Sketch.png":
        NoneType = type(im)
    
        break

count_none = 0
  
width = 420
height = 240

for index, row in data.iterrows():
    # print(row["filename"], row["X"], row["subject"])
    
    im = cv2.imread(row["filename"])
    
    if type(im) != None:
        
        im = cv2.resize(im, (height, width), interpolation = cv2.INTER_CUBIC)
        print(row["filename"])
        vec = np.array(np.reshape(im, (-1, 1)))
        
        data['X'][index] = vec
        
        """
        if ((data1['filename'] == row['filename']) & (data1['sketch'] == 0)).any():
            if index == 0:
                X = vec
                y_subject = np.array(data['subject'][index])

            else:
                X = np.concatenate((X, vec), axis=1)
                y_subject = np.concatenate((y_subject, np.array(data['subject'][index])), axis=1)
        """
        
    if type(im) == None:
        count_none += 1
        print(row["filename"])
    
print(count_none)

# Let's print the first ten values for the signal vector
print(data.head())

/root/ML
1641002_Male_Anger_RGB.png
1641002_Male_Angry_Sketch.png


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



1641002_Male_Disgust_RGB.png
1641002_Male_Disgust_Sketch.png
1641002_Male_Fear_RGB.png
1641002_Male_Fear_Sketch.png
1641002_Male_Happy_RGB.png
1641002_Male_Happy_Sketch.png
1641002_Male_Neutral_RGB.png
1641002_Male_Neutral_Sketch.png
1641002_Male_Sad_RGB.png
1641002_Male_Sad_Sketch.png
1641002_Male_Surprise_RGB.png
1641002_Male_Surprise_Sketch.png
1641003_Male_Disgust_RGB.png
1641003_Male_Disgust_Sketch.png
1641003_Male_Happy_RGB.png
1641003_Male_Happy_Sketch.png
1641003_Male_Neutral_RGB.png
1641003_Male_Neutral_Sketch.png
1641003_Male_Sad_RGB.png
1641003_Male_Sad_Sketch.png
1641003_Male_surprise_RGB.png
1641003_Male_surprise_Sketch.png
1641005_Male_Anger_RGB.png
1641005_Male_Anger_Sketch.png
1641005_Male_Disgust_RGB.png
1641005_Male_Disgust_Sketch.png
1641005_Male_Fear_RGB.png
1641005_Male_Fear_Sketch.png
1641005_Male_Happy_RGB.png
1641005_Male_Happy_Sketch.png
1641005_Male_Neutral_RGB.png
1641005_Male_Neutral_Sketch.png
1641005_Male_Sad_RGB.png
1641005_Male_Sad_Sketch.png
1641005_Mal

In [0]:
data = pd.merge(data, data1, on=['filename'])
data = pd.merge(data, data2, on=['filename'])
data = pd.merge(data, data3, on=['filename'])

In [0]:
# Deleting None rows
print(data.shape)
data = data.dropna()
print(data.shape)
data


(860, 3)
(860, 3)


,filename,subject,X
0,1641002_Male_Anger_RGB.png,0,"[[186], [197], [201], [192], [198], [203], [19..."
1,1641002_Male_Angry_Sketch.png,0,"[[253], [253], [253], [253], [253], [253], [25..."
2,1641002_Male_Disgust_RGB.png,0,"[[185], [197], [201], [186], [199], [201], [18..."
3,1641002_Male_Disgust_Sketch.png,0,"[[254], [254], [254], [252], [252], [252], [25..."
4,1641002_Male_Fear_RGB.png,0,"[[188], [197], [201], [187], [198], [202], [18..."
5,1641002_Male_Fear_Sketch.png,0,"[[253], [253], [253], [253], [253], [253], [25..."
6,1641002_Male_Happy_RGB.png,0,"[[196], [205], [208], [195], [204], [207], [19..."
7,1641002_Male_Happy_Sketch.png,0,"[[253], [253], [253], [254], [254], [254], [25..."
8,1641002_Male_Neutral_RGB.png,0,"[[194], [202], [202], [187], [197], [197], [19..."
9,1641002_Male_Neutral_Sketch.png,0,"[[253], [253], [253], [252], [252], [252], [25..."


In [0]:
data_nosketch = data[data['sketch'] == 0]
data_nosketch

In [0]:
def return_dataset(data, class_name):
    for index, row in data.iterrows():
        
        vec = data['X'][index]
      
        if index == 0:
            X = vec
            y = np.reshape(data[class_name][index], (1, 1))

        else:
            X = np.concatenate((X, vec), axis=1)
            y = np.concatenate((y, np.reshape(data[class_name][index], (1, 1))), axis=0)
            
    return X, y

In [0]:
X, y = return_dataset(data, 'subject')
X = X.T

# print(X.shape)

#train_x, test_x, train_y, test_y = model_selection.train_test_split(X, y, test_size = 0.1, random_state = 0)

#X = X.astype(np.float32)
#X = X.T
#print(type(X))

X_train_DT, X_test_DT, y_train_DT, y_test_DT = model_selection.train_test_split(X, data['subject'], test_size = 0.1, random_state = 0)
X_train_KNN, X_test_KNN, y_train_KNN, y_test_KNN = model_selection.train_test_split(X, data['subject'], test_size = 0.1, random_state = 0)
X_train_SVM, X_test_SVM, y_train_SVM, y_test_SVM = model_selection.train_test_split(X, data['subject'], test_size = 0.1, random_state = 0)
X_train_RF, X_test_RF, y_train_RF, y_test_RF = model_selection.train_test_split(X, data['subject'], test_size = 0.1, random_state = 0)
X_train_NB, X_test_NB, y_train_NB, y_test_NB = model_selection.train_test_split(X, data['subject'], test_size = 0.1, random_state = 0)

In [0]:
from sklearn.tree import DecisionTreeClassifier
classifier_DT = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_DT.fit(X_train_DT, y_train_DT)


In [0]:
from sklearn.neighbors import KNeighborsClassifier
classifier_KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_KNN.fit(X_train_KNN, y_train_KNN)


In [0]:
from sklearn.svm import SVC
classifier_SVM = SVC(kernel = 'linear', random_state = 0)
classifier_SVM.fit(X_train_SVM, y_train_SVM)


In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_NB = sc.fit_transform(X_train_NB)
X_test_NB = sc.transform(X_test_NB)

from sklearn.naive_bayes import GaussianNB
classifier_NB = GaussianNB()
classifier_NB.fit(X_train_NB, y_train_NB)



In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier_RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_RF.fit(X_train_RF, y_train_RF)


In [0]:
y_pred_DT = classifier.predict(X_test_DT)
y_test_DT = y_test_DT.tolist() 

y_pred_SVM = classifier.predict(X_test_SVM)
y_test_SVM = y_test_SVM.tolist() 

y_pred_RF = classifier.predict(X_test_RF)
y_test_RF = y_test_RF.tolist() 

y_pred_NB = classifier.predict(X_test_NB)
y_test_NB = y_test_NB.tolist() 

y_pred_KNN = classifier.predict(X_test_KNN)
y_test_KNN = y_test_KNN.tolist() 



In [0]:
y_test_np_DT = np.asarray(y_test_DT)
y_test_np_SVM = np.asarray(y_test_SVM)
y_test_np_RF = np.asarray(y_test_RF)
y_test_np_KNN = np.asarray(y_test_KNN)
y_test_np_NB = np.asarray(y_test_NB)



In [0]:
# Accuracy
##############################################
disc_NB = y_test_np_NB - y_pred_NB

count = 0
for i in disc_NB:
  if i == 0:
    count += 1

accuracy_NB = ( (100 * count) / len(y_pred_NB))
##############################################
disc_RF = y_test_np_RF - y_pred_RF

count = 0
for i in disc_RF:
  if i == 0:
    count += 1

accuracy_RF = ( (100 * count) / len(y_pred_RF))
##############################################
disc_KNN = y_test_np_KNN - y_pred_KNN

count = 0
for i in disc_KNN:
  if i == 0:
    count += 1

accuracy_KNN = ( (100 * count) / len(y_pred_KNN))
##############################################
disc_SVM = y_test_np_SVM - y_pred_SVM

count = 0
for i in disc_SVM:
  if i == 0:
    count += 1

accuracy_SVM = ( (100 * count) / len(y_pred_SVM))
##############################################
disc_DT = y_test_np_DT - y_pred_DT

count = 0
for i in disc_DT:
  if i == 0:
    count += 1

accuracy_DT = ( (100 * count) / len(y_pred_DT))
##############################################



print(accuracy_NB)
print(accuracy_RF)
print(accuracy_DT)
print(accuracy_KNN)
print(accuracy_SVM)